### Modules

In [1]:
import archives_manager
import board_feature_extractor
import game_state_classifier_data_entry

### Gathering data

In [2]:
# Set player name and amount of games you want to get
player_name = "BIG_TONKA_T"
game_count = 50

# Begin loading game data into json
game_state_classifier_data_entry.data_entry(player_name, game_count)

This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This game has already been entered. Skipping duplicate entry.
This gam

### Compiling data into dataframes

In [3]:
import pandas as pd
import json
import chess
import chess.pgn
from chess import Board
import io


# Load the JSON data from the file
with open('game_state_classifier_data.json', 'r') as file:
    data = json.load(file)

# Prepare dataset
rows = []

for game in data['games']:
    pgn = game['pgn']
    move_count_middle_game = game['move_count_middle_game']
    move_count_end_game = game['move_count_end_game']

    game_obj = chess.pgn.read_game(io.StringIO(pgn))
    board = game_obj.board()

    for move in game_obj.mainline_moves():
        board.push(move)
        move_count = board.fullmove_number - 0.5 # Starts at 0.5 instead of 1
        if board.turn == chess.WHITE:  # Adjust for half moves (this checks whose turn is next, not the current turn)
            move_count -= 0.5 # subtracts 0.5 from black

        pieces_count = board_feature_extractor.count_pieces(board)
        developed_pieces_count = board_feature_extractor.count_developed_pieces(board)
        open_files_count = board_feature_extractor.count_open_files(board)
        passed_pawns_count = board_feature_extractor.count_passed_pawns(board)

        if move_count < move_count_middle_game:
            game_state = 'OPEN'
        elif move_count < move_count_end_game:
            game_state = 'MID'
        else:
            game_state = 'END'

        row = {
            'move_count': move_count,
            'white_pawns': pieces_count['pawns']['white'],
            'black_pawns': pieces_count['pawns']['black'],
            'white_pieces': pieces_count['pieces']['white'],
            'black_pieces': pieces_count['pieces']['black'],
            'white_developed_pieces': developed_pieces_count['white'],
            'black_developed_pieces': developed_pieces_count['black'],
            'open_files': open_files_count,
            'white_passed_pawns': passed_pawns_count['white'],
            'black_passed_pawns': passed_pawns_count['black'],
            'game_state': game_state
        }
        rows.append(row)

df = pd.DataFrame(rows)
df.head(20)

,move_count,white_pawns,black_pawns,white_pieces,black_pieces,white_developed_pieces,black_developed_pieces,open_files,white_passed_pawns,black_passed_pawns,game_state
0,0.5,8,8,7,7,0,0,0,0,0,OPEN
1,1.0,8,8,7,7,0,1,0,0,0,OPEN
2,1.5,8,8,7,7,1,1,0,0,0,OPEN
3,2.0,8,8,7,7,1,1,0,0,0,OPEN
4,2.5,8,8,7,7,2,1,0,0,0,OPEN
5,3.0,8,8,7,7,2,1,0,0,0,OPEN
6,3.5,8,8,7,7,3,1,0,0,0,OPEN
7,4.0,8,8,7,7,3,2,0,0,0,OPEN
8,4.5,8,8,7,7,3,2,0,0,0,OPEN
9,5.0,8,8,7,7,3,2,0,0,0,OPEN


### Building Classifier

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Splitting the data into features and target variable
X = df.drop('game_state', axis=1)  # Features
y = df['game_state']  # Target variable

# Splitting the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the classifier using the training data
knn.fit(X_train, y_train)

KNeighborsClassifier()

### Testing Classifier

In [5]:
# Predict the labels for the test set
y_pred = knn.predict(X_test)

# Evaluate the classifier
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[110   1   0]
 [  3 246   4]
 [  0   4 169]]

Classification Report:
              precision    recall  f1-score   support

         END       0.97      0.99      0.98       111
         MID       0.98      0.97      0.98       253
        OPEN       0.98      0.98      0.98       173

    accuracy                           0.98       537
   macro avg       0.98      0.98      0.98       537
weighted avg       0.98      0.98      0.98       537

Accuracy Score: 0.9776536312849162
